# Lecture du fichier

In [ ]:
import math
import time
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def read_tsp_file(file_path):
    coordinates = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        node_coord_section_index = lines.index('NODE_COORD_SECTION\n')
        for line in lines[node_coord_section_index + 1:]:
            if line.strip() == 'EOF':
                break
            parts = line.strip().split()
            node_id = int(parts[0])
            x, y = map(float, parts[1:])
            coordinates[node_id] = (x, y)
    return coordinates


def create_distance_matrix(coordinates):
    num_cities = len(coordinates)
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distance_matrix[i - 1][j - 1] = calculate_distance(coordinates[i], coordinates[j])

    return distance_matrix

# Example usage
file_path = "eil101.tsp"
coordinates = read_tsp_file(file_path)
distances = create_distance_matrix(coordinates)


# ***Ant Colony Optimization***

ACO (Ant Colony Optimization) est une technique métaheuristique inspirée du
comportement des fourmis cherchant des chemins optimaux entre leur colonie et une source de nourriture. Il est souvent utilisé pour résoudre des problèmes d'optimisation combinatoire, tels que le problème du voyageur de commerce (PVC).

L'algorithme ACO fonctionne en modélisant le comportement des fourmis qui laissent des phéromones sur les chemins qu'elles parcourent. Les chemins avec une concentration plus élevée de phéromones sont plus attrayants pour les autres fourmis, ce qui conduit éventuellement à la découverte de chemins optimaux. Voici une vue d'ensemble de son fonctionnement :

* **Initialisation des fourmis** : Un nombre de fourmis artificielles est placé dans l'espace de recherche.

* **Construction de solutions** : Chaque fourmi construit itérativement une solution en sélectionnant des chemins basés sur des règles probabilistes.
* **Dépôt de phéromones** : Après avoir complété son chemin, chaque fourmi dépose des phéromones sur les chemins empruntés.
* **Évaporation des phéromones** : Les phéromones s'évaporent à chaque itération, permettant d'éliminer les chemins peu prometteurs.
* **Exploration et exploitation** : Les fourmis explorent de nouvelles régions tout en exploitant les chemins les plus prometteurs, guidées par les phéromones.
* **Critère d'arrêt** : L'algorithme s'arrête lorsqu'un critère prédéfini est atteint, comme un nombre fixe d'itérations ou l'absence d'amélioration significative.
* **Sélection de la meilleure solution** : La meilleure solution trouvée parmi toutes les solutions construites est sélectionnée à la fin de l'algorithme.

###Dans ce qui suit nous allons expliqué chaque variante et détaillé les étapes

## ACO simple
Voici une description détaillée des différentes étapes de l'implémentation de l'algorithme ACO fournie :

* **Initialisation des phéromones** : La fonction initialize_pheromones initialise une matrice de phéromones de taille num_cities x num_cities avec une valeur initiale v_init.

* **Algorithme ACO** :

 1. Itérations : L'algorithme effectue un certain nombre d'itérations déterminé par num_iterations.

 2. Initialisation locale des phéromones : Pour chaque itération, une matrice locale de phéromones pheromones_local est initialisée avec des valeurs nulles.
 3. Parcours des fourmis : Pour chaque fourmi dans num_ants, un parcours est effectué.
 4. Choix initial de la ville : Une ville de départ est choisie aléatoirement.
 5. Construction du chemin : La fourmi parcourt les villes restantes en choisissant la prochaine ville à visiter en fonction de probabilités calculées à partir des phéromones et des distances entre les villes.
 6. Mise à jour du chemin et des visites : Le chemin est mis à jour avec la ville choisie, et la ville est ajoutée à l'ensemble des villes visitées.
 7. Calcul de la distance du chemin parcouru : Une fois que toutes les villes ont été visitées, la distance totale du chemin est calculée.
 8. Mise à jour de la meilleure solution : Si la distance de ce chemin est meilleure que celle de la meilleure solution actuelle, la meilleure solution est mise à jour.
 9. Mise à jour des phéromones locales : Les phéromones locales sont mises à jour en fonction de la qualité du chemin parcouru par la fourmi.
 10. Mise à jour globale des phéromones : Une fois que toutes les fourmis ont terminé leur parcours, les phéromones globales sont mises à jour en combinant les phéromones locales et en appliquant un taux d'évaporation.

In [3]:
import random

# Fonction pour calculer la longueur du chemin
def tour_length(tour, distances):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distances[tour[i]][tour[i+1]]
    return total_distance

# Fonction pour initialiser les phéromones sur chaque chemin
def initialize_pheromones(num_cities,v_init):
    pheromones = [[v_init] * num_cities for _ in range(num_cities)]
    return pheromones

# Algorithme ACO
def ant_colony_optimization(distances, num_ants, num_iterations,pheromones, alpha, beta, Q, evaporation_rate):
    num_cities = len(distances)
    best_solution = None
    best_distance = float('inf')

    for _ in range(num_iterations):
        pheromones_local= initialize_pheromones(num_cities, 0.0)
        for ant in range(num_ants):
            current_city = random.randint(0, num_cities - 1)
            tour = [current_city]
            visited = {current_city}

            while len(visited) < num_cities:
                probabilities = []
                total_pheromone = sum(pheromones[current_city][neighbor] ** alpha * (1 / distances[current_city][neighbor]) ** beta for neighbor in range(num_cities) if neighbor not in visited)
                for neighbor in range(num_cities):
                    if neighbor not in visited:
                        pheromone = pheromones[current_city][neighbor]
                        probability = (pheromone ** alpha) * ((1 / distances[current_city][neighbor]) ** beta) / total_pheromone
                        #print(neighbor,probability)
                        probabilities.append((neighbor, probability))

                next_city = random.choices([neighbor for neighbor, _ in probabilities], [prob for _, prob in probabilities])[0]
                tour.append(next_city)
                visited.add(next_city)
                current_city = next_city

            tour.append(tour[0])
            tour_dist = tour_length(tour, distances)

            if tour_dist < best_distance:
                best_solution = tour
                best_distance = tour_dist

            for i in range(len(tour) - 1):
                pheromones_local[tour[i]][tour[i+1]] += Q / tour_dist

        for i in range(num_cities):
            for j in range(num_cities):
                pheromones[i][j] = (1 - evaporation_rate) * pheromones[i][j] + pheromones_local[i][j]

    return best_solution, best_distance

# Exemple d'utilisation
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
num_cities = len(distances)


num_ants = 75
num_iterations = 15
pheromones = initialize_pheromones(num_cities, 0.3)
alpha = 5.0
beta = 3.0
Q = 1
evaporation_rate = 0.2

best_solution, best_distance = ant_colony_optimization(distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate)
print("Meilleure solution trouvée:", best_solution)
print("Longueur du meilleur chemin:", best_distance)

Meilleure solution trouvée: [3, 4, 2, 0, 1, 3]
Longueur du meilleur chemin: 65


## Variante AS et Elitisme

Dans cette version modifiée de l'algorithme ACO, nous avons ajouté une stratégie appelée "Élitisme", où la meilleure solution trouvée parmi toutes les fourmis lors d'une itération est privilégiée dans la mise à jour des phéromones locales.

Voici un résumé des modifications apportées :

* **Initialisation de variables** : Une variable best_ant_tour est introduite pour stocker le meilleur parcours trouvé parmi toutes les fourmis lors d'une itération, ainsi que sa distance (best_ant_distance).
* **Boucle des fourmis** :
 * Après la construction de chaque parcours par une fourmi, on vérifie si ce parcours est meilleur que le meilleur parcours actuel (best_ant_distance).
 * Si c'est le cas, le meilleur parcours est mis à jour avec le parcours de la fourmi actuelle.
* **Mise à jour des phéromones locales** : Après la construction de tous les parcours par les fourmis, les phéromones locales sont mises à jour normalement en incluant une 2eme fois le meilleur parcours trouvé (best_ant_tour).
* **Évaporation et mise à jour globale des phéromones** : Les phéromones locales sont ensuite combinées avec les phéromones globales, en appliquant le taux d'évaporation.

In [ ]:
import random

# Fonction pour calculer la longueur du chemin
def tour_length(tour, distances):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distances[tour[i]][tour[i+1]]
    return total_distance

# Fonction pour initialiser les phéromones sur chaque chemin
def initialize_pheromones(num_cities,v_init):
    pheromones = [[v_init] * num_cities for _ in range(num_cities)]
    return pheromones

# Algorithme ACO
def ant_colony_optimization(distances, num_ants, num_iterations,pheromones, alpha, beta, Q, evaporation_rate):
    num_cities = len(distances)
    best_solution = None
    best_distance = float('inf')

    for _ in range(num_iterations):
        pheromones_local= initialize_pheromones(num_cities, 0.0)
        #ant_solutions=[]   #Solution 2
        best_ant_tour = None
        best_ant_distance = float('inf')

        for ant in range(num_ants):
            current_city = random.randint(0, num_cities - 1)
            tour = [current_city]
            visited = {current_city}

            while len(visited) < num_cities:
                probabilities = []
                total_pheromone = sum(pheromones[current_city][neighbor] ** alpha * (1 / distances[current_city][neighbor]) ** beta for neighbor in range(num_cities) if neighbor not in visited)
                for neighbor in range(num_cities):
                    if neighbor not in visited:
                        pheromone = pheromones[current_city][neighbor]
                        probability = (pheromone ** alpha) * ((1 / distances[current_city][neighbor]) ** beta) / total_pheromone
                        probabilities.append((neighbor, probability))

                next_city = random.choices([neighbor for neighbor, _ in probabilities], [prob for _, prob in probabilities])[0]
                tour.append(next_city)
                visited.add(next_city)
                current_city = next_city

            tour.append(tour[0])
            tour_dist = tour_length(tour, distances)

            if tour_dist < best_distance:
                best_solution = tour
                best_distance = tour_dist

            if tour_dist < best_ant_distance :
                best_ant_distance = tour_dist
                best_ant_tour = tour

            #ant_solutions.append((tour, tour_dist))       #Solution 2

            for i in range(len(tour) - 1):
                pheromones_local[tour[i]][tour[i+1]] += Q / tour_dist

        #best_ant_tour, best_ant_distance = min(ant_solutions, key=lambda x: x[1])        #Solution 2
        for i in range(len(best_ant_tour) - 1):
            pheromones_local[best_ant_tour[i]][best_ant_tour[i+1]] += Q / best_ant_distance

        for i in range(num_cities):
            for j in range(num_cities):
                pheromones[i][j] = (1 - evaporation_rate) * pheromones[i][j] + pheromones_local[i][j]
    #print(pheromones)

    return best_solution, best_distance

# Exemple d'utilisation
"""
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
"""
num_cities = len(distances)


num_ants = 75
num_iterations = 15
pheromones = initialize_pheromones(num_cities, 0.3)
alpha = 5.0
beta = 4.0
Q = 1
evaporation_rate = 0.2

best_solution, best_distance = ant_colony_optimization(distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate)
print("Meilleure solution trouvée:", best_solution)
print("Longueur du meilleur chemin:", best_distance)

Meilleure solution trouvée: [47, 46, 35, 48, 63, 10, 18, 61, 9, 89, 31, 62, 29, 69, 30, 87, 6, 81, 7, 44, 16, 83, 4, 59, 82, 17, 51, 88, 5, 93, 94, 96, 86, 1, 56, 14, 42, 41, 13, 37, 85, 43, 15, 60, 84, 92, 98, 95, 58, 91, 36, 97, 99, 90, 12, 57, 39, 20, 72, 71, 73, 21, 40, 74, 55, 22, 38, 66, 24, 54, 53, 3, 23, 28, 67, 76, 2, 78, 77, 33, 34, 70, 65, 64, 8, 50, 19, 80, 32, 49, 0, 68, 26, 100, 52, 27, 25, 11, 79, 75, 45, 47]
Longueur du meilleur chemin: 735.9607263953799


## MMAS
Dans cette version modifiée de l'algorithme ACO, nous avons inclus une fonctionnalité pour contrôler les niveaux de phéromones déposés en utilisant des valeurs minimales et maximales, appelées min_pheromone et max_pheromone.
De plus dans un iteration sauf la fourmi ayant trouver le meilleur chemin mis a jour de pheromone.

In [ ]:
import random

# Fonction pour calculer la longueur du chemin
def tour_length(tour, distances):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distances[tour[i]][tour[i+1]]
    return total_distance

# Fonction pour initialiser les phéromones sur chaque chemin
def initialize_pheromones(num_cities,v_init):
    pheromones = [[v_init] * num_cities for _ in range(num_cities)]
    return pheromones

# Algorithme ACO
def ant_colony_optimization(distances, num_ants, num_iterations,pheromones, alpha, beta, Q, evaporation_rate, min_pheromone, max_pheromone):
    num_cities = len(distances)
    best_solution = None
    best_distance = float('inf')

    for _ in range(num_iterations):
        pheromones_local= initialize_pheromones(num_cities, 0.0)
        best_ant_tour = None
        best_ant_distance = float('inf')

        for ant in range(num_ants):
            current_city = random.randint(0, num_cities - 1)
            tour = [current_city]
            visited = {current_city}

            while len(visited) < num_cities:
                probabilities = []
                total_pheromone = sum(pheromones[current_city][neighbor] ** alpha * (1 / distances[current_city][neighbor]) ** beta for neighbor in range(num_cities) if neighbor not in visited)
                for neighbor in range(num_cities):
                    if neighbor not in visited:
                        pheromone = pheromones[current_city][neighbor]
                        probability = (pheromone ** alpha) * ((1 / distances[current_city][neighbor]) ** beta) / total_pheromone
                        probabilities.append((neighbor, probability))

                next_city = random.choices([neighbor for neighbor, _ in probabilities], [prob for _, prob in probabilities])[0]
                tour.append(next_city)
                visited.add(next_city)
                current_city = next_city

            tour.append(tour[0])
            tour_dist = tour_length(tour, distances)

            if tour_dist < best_distance:
                best_solution = tour
                best_distance = tour_dist

            if tour_dist < best_ant_distance :
                best_ant_distance = tour_dist
                best_ant_tour = tour


        for i in range(len(best_ant_tour) - 1):
            pheromones_local[best_ant_tour[i]][best_ant_tour[i+1]] += Q / best_ant_distance

        for i in range(num_cities):
            for j in range(num_cities):
                pheromones[i][j] = (1 - evaporation_rate) * pheromones[i][j] + pheromones_local[i][j]
                pheromones[i][j] = max(min_pheromone, min(max_pheromone, pheromones[i][j]))


    return best_solution, best_distance

# Exemple d'utilisation
"""
distances = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 15, 10],
    [20, 25, 15, 0, 5],
    [25, 30, 10, 5, 0]
]
"""
num_cities = len(distances)

min_pheromone = 0.00000001
max_pheromone = 1.0
num_ants = 101
num_iterations = 20
pheromones = initialize_pheromones(num_cities, min_pheromone)
alpha = 1.0
beta = 2.0
Q = 1
evaporation_rate = 0.5


best_solution, best_distance = ant_colony_optimization(distances, num_ants, num_iterations, pheromones, alpha, beta, Q, evaporation_rate, min_pheromone, max_pheromone)
print("Meilleure solution trouvée:", best_solution)
print("Longueur du meilleur chemin:", best_distance)

Meilleure solution trouvée: [22, 23, 19, 65, 64, 50, 92, 12, 34, 70, 31, 94, 5, 37, 15, 7, 88, 74, 40, 57, 4, 76, 69, 8, 33, 62, 18, 46, 81, 51, 61, 6, 60, 36, 84, 59, 17, 82, 43, 13, 90, 14, 79, 67, 54, 27, 21, 66, 75, 26, 100, 78, 2, 87, 89, 93, 95, 85, 99, 77, 80, 32, 48, 96, 16, 83, 58, 91, 97, 56, 86, 71, 28, 52, 39, 38, 72, 20, 42, 35, 0, 11, 9, 47, 10, 29, 73, 41, 3, 25, 1, 98, 68, 63, 44, 45, 30, 55, 49, 24, 53, 22]
Longueur du meilleur chemin: 2190.3038490915833
